# Тестовое задание 
### Выполнил: Линдеманн Никита

Может помочь для решения задачи:
- https://github.com/scikit-learn-contrib/metric-learn
- https://arxiv.org/pdf/1503.03832.pdf
- https://github.com/KevinMusgrave/pytorch-metric-learning
- https://arxiv.org/pdf/2003.08505.pdf

### 1. Baseline

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("train_data.csv")
df.shape

(20952, 3)

In [3]:
df.head(10)

,posting_id,title,label_group
0,train_129225211,Paper Bag Victoria Secret,249114794
1,train_2278313361,PAPER BAG VICTORIA SECRET,249114794
2,train_2288590299,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,train_3803689425,Maling Ham Pork Luncheon Meat TTS 397gr,2395904891
4,train_2406599165,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188
5,train_3342059966,DASTER PIYAMA KATUN JEPANG(TIDAK BISA PILIH MO...,4093212188
6,train_1806152124,KULOT PLISKET SALUR /CANDY PLISKET /WISH KULOT...,1565741687
7,train_3227306976,KULOT PLISKET MOMBLU,1565741687
8,train_86570404,"[LOGU] Tempelan kulkas magnet angka, tempelan ...",2359912463
9,train_2837452969,"10Pcs Magnet Kulkas Model Angka 0-9, Bahan Kay...",2359912463


In [4]:
df.label_group.unique().shape

(6608,)

In [5]:
tmp = df.groupby(['label_group'])['posting_id'].unique().to_dict()
df['matches'] = df['label_group'].map(tmp)
df['matches'] = df['matches'].apply(lambda x: ' '.join(x))

Нашей метрикой будет построчный f1:
$$
F_1 = 2 \cdot \frac{Precision \cdot Recall}{Precision + Recall} = \frac{2 \cdot TP}{2 \cdot TP + FN + FP}.
$$

In [6]:
def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1

в качестве бейзлайна сопоставим себя в группу только с самим собой и каким нибудь еще одним случайным товаром

In [7]:
df['prediction_0'] = df['posting_id'] + ' ' + 'train_129225211'

In [8]:
f1_score(df['matches'], df['prediction_0']).mean()

0.35675992543848534

In [9]:
df.head(10)

,posting_id,title,label_group,matches,prediction_0
0,train_129225211,Paper Bag Victoria Secret,249114794,train_129225211 train_2278313361,train_129225211 train_129225211
1,train_2278313361,PAPER BAG VICTORIA SECRET,249114794,train_129225211 train_2278313361,train_2278313361 train_129225211
2,train_2288590299,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,train_2288590299 train_3803689425,train_2288590299 train_129225211
3,train_3803689425,Maling Ham Pork Luncheon Meat TTS 397gr,2395904891,train_2288590299 train_3803689425,train_3803689425 train_129225211
4,train_2406599165,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,train_2406599165 train_3342059966,train_2406599165 train_129225211
5,train_3342059966,DASTER PIYAMA KATUN JEPANG(TIDAK BISA PILIH MO...,4093212188,train_2406599165 train_3342059966,train_3342059966 train_129225211
6,train_1806152124,KULOT PLISKET SALUR /CANDY PLISKET /WISH KULOT...,1565741687,train_1806152124 train_3227306976,train_1806152124 train_129225211
7,train_3227306976,KULOT PLISKET MOMBLU,1565741687,train_1806152124 train_3227306976,train_3227306976 train_129225211
8,train_86570404,"[LOGU] Tempelan kulkas magnet angka, tempelan ...",2359912463,train_86570404 train_2837452969 train_77364776,train_86570404 train_129225211
9,train_2837452969,"10Pcs Magnet Kulkas Model Angka 0-9, Bahan Kay...",2359912463,train_86570404 train_2837452969 train_77364776,train_2837452969 train_129225211


Твоя цель улучшить эту метрику, можешь пользоваться любыми методами, но учитывай что на проде данных много и методы должны адаптироваться на большие данные.

### 2. Native

Попробуем простой подход: проведем обработку текста (препроцессинг, токенизация, стемминг/лемматизация) и будем матчить объекты путем поэлементного сравнения токенов.

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.cluster import AgglomerativeClustering

import re
import nltk
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [35]:
def clean_text(text):
    """
    Source: https://webtort.ru/%D0%BF%D1%80%D0%B5%D0%BF%D1%80%D0%BE%D1%86%D0%B5%D1%81%D1%81%D0%B8%D0%BD%D0%B3-%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0-text-preprocessing-%D0%B8%D0%BB%D0%B8-%D0%B3%D0%BE%D1%82%D0%BE%D0%B2%D0%B8%D0%BC/
    """
    # приводим текст к нижнему регистру
    text = text.lower()
    # создаем регулярное выражение для удаления лишних символов
    regular = r'[\*+\#+\№\"\-+\+\=+\?+\&\^\.+\;\,+\>+\(\)\/+\:\\+\]+\[+|+!+%+@]'
    # регулярное выражение для замены ссылки на "URL"
    regular_url = r'(http\S+)|(www\S+)|([\w\d]+www\S+)|([\w\d]+http\S+)'
    # удаляем лишние символы
    text = re.sub(regular, '', text)
    # заменяем ссылки на "URL"
    text = re.sub(regular_url, r'URL', text)
    # заменяем числа и цифры на ' NUM '
    text = re.sub(r'(\d+\s\d+)|(\d+)',' NUM ', text) # возможно, стоит удалить?
    # удаляем лишние пробелы
    text = re.sub(r'\s+', ' ', text)
    # возвращаем очищенные данные
    return text

In [36]:
processed_text = []
tokens = []
# стоп-слова 
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

for posting_id, text in zip(df['posting_id'], df['title']):
    text_0 = text
    text = clean_text(text)   
    text = word_tokenize(text)       
    # удаление стоп-слов
    text = [word for word in text if word not in stop_words]     
    # лемматизация
    text = [lemmatizer.lemmatize(w) for w in text]
    tokens.append(text)
    processed_text.append((posting_id, text_0, text))
    
processed_text # (id, defualt_text, tokenized_text)

[('train_129225211',
  'Paper Bag Victoria Secret',
  ['paper', 'bag', 'victoria', 'secret']),
 ('train_2278313361',
  'PAPER BAG VICTORIA SECRET',
  ['paper', 'bag', 'victoria', 'secret']),
 ('train_2288590299',
  'Maling TTS Canned Pork Luncheon Meat 397 gr',
  ['maling', 'tt', 'canned', 'pork', 'luncheon', 'meat', 'NUM', 'gr']),
 ('train_3803689425',
  'Maling Ham Pork Luncheon Meat TTS 397gr',
  ['maling', 'ham', 'pork', 'luncheon', 'meat', 'tt', 'NUM', 'gr']),
 ('train_2406599165',
  'Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi',
  ['daster',
   'batik',
   'lengan',
   'pendek',
   'motif',
   'acak',
   'campur',
   'leher',
   'kancing',
   'dpt',
   'NUM',
   'batik',
   'karakter',
   'alhadi']),
 ('train_3342059966',
  'DASTER PIYAMA KATUN JEPANG(TIDAK BISA PILIH MOTIF & WARNA)',
  ['daster',
   'piyama',
   'katun',
   'jepangtidak',
   'bisa',
   'pilih',
   'motif',
   'warna']),
 ('train_1806152124',
  'KULOT PLISKET

In [37]:
df['tokens'] = tokens

In [40]:
result = {}

for min_threshold in range(2, 8):
    print(20*"--", min_threshold, 20*"--")
    pred = []
    
    for row in df.itertuples():
        if row[0] % 100 == 0:
            print("Num: ", row[0])
        label = []
        try:
            for row_next in df[int(row[0])+1:].itertuples():
                if len(set(row[6])&set(row_next[6])) >= min_threshold:
                    label.append(row_next[4])
            if len(label) != 0:
                pred.append(" ".join(i for i in label))
            else:
                pred.append(" ")
        except IndexError:
            print("IndexError")

    df["prediction_1"] = pred
    result[min_threshold] = f1_score(df['matches'], df['prediction_1']).mean()
    
# df.to_csv("native.csv", index=False)

---------------------------------------- 2 ----------------------------------------
Num:  0
Num:  100
Num:  200
Num:  300
Num:  400
Num:  500
Num:  600
Num:  700
Num:  800
Num:  900
Num:  1000
Num:  1100
Num:  1200
Num:  1300
Num:  1400
Num:  1500
Num:  1600
Num:  1700
Num:  1800
Num:  1900
Num:  2000
Num:  2100
Num:  2200
Num:  2300
Num:  2400
Num:  2500
Num:  2600
Num:  2700
Num:  2800
Num:  2900
Num:  3000
Num:  3100
Num:  3200
Num:  3300
Num:  3400
Num:  3500
Num:  3600
Num:  3700
Num:  3800
Num:  3900
Num:  4000
Num:  4100
Num:  4200
Num:  4300
Num:  4400
Num:  4500
Num:  4600
Num:  4700
Num:  4800
Num:  4900
Num:  5000
Num:  5100
Num:  5200
Num:  5300
Num:  5400
Num:  5500
Num:  5600
Num:  5700
Num:  5800
Num:  5900
Num:  6000
Num:  6100
Num:  6200
Num:  6300
Num:  6400
Num:  6500
Num:  6600
Num:  6700
Num:  6800
Num:  6900
Num:  7000
Num:  7100
Num:  7200
Num:  7300
Num:  7400
Num:  7500
Num:  7600
Num:  7700
Num:  7800
Num:  7900
Num:  8000
Num:  8100
Num:  8200
Num:  8300
Num:

Num:  5900
Num:  6000
Num:  6100
Num:  6200
Num:  6300
Num:  6400
Num:  6500
Num:  6600
Num:  6700
Num:  6800
Num:  6900
Num:  7000
Num:  7100
Num:  7200
Num:  7300
Num:  7400
Num:  7500
Num:  7600
Num:  7700
Num:  7800
Num:  7900
Num:  8000
Num:  8100
Num:  8200
Num:  8300
Num:  8400
Num:  8500
Num:  8600
Num:  8700
Num:  8800
Num:  8900
Num:  9000
Num:  9100
Num:  9200
Num:  9300
Num:  9400
Num:  9500
Num:  9600
Num:  9700
Num:  9800
Num:  9900
Num:  10000
Num:  10100
Num:  10200
Num:  10300
Num:  10400
Num:  10500
Num:  10600
Num:  10700
Num:  10800
Num:  10900
Num:  11000
Num:  11100
Num:  11200
Num:  11300
Num:  11400
Num:  11500
Num:  11600
Num:  11700
Num:  11800
Num:  11900
Num:  12000
Num:  12100
Num:  12200
Num:  12300
Num:  12400
Num:  12500
Num:  12600
Num:  12700
Num:  12800
Num:  12900
Num:  13000
Num:  13100
Num:  13200
Num:  13300
Num:  13400
Num:  13500
Num:  13600
Num:  13700
Num:  13800
Num:  13900
Num:  14000
Num:  14100
Num:  14200
Num:  14300
Num:  14400
Num:  145

In [41]:
result

{2: 0.10829333555558385,
 3: 0.24937290330268977,
 4: 0.34791253733002075,
 5: 0.3381139321780553,
 6: 0.27202723522926553}

In [43]:
df

,posting_id,title,label_group,matches,prediction_0,tokens,prediction_1
0,train_129225211,Paper Bag Victoria Secret,249114794,train_129225211 train_2278313361,train_129225211 train_129225211,"[paper, bag, victoria, secret]",
1,train_2278313361,PAPER BAG VICTORIA SECRET,249114794,train_129225211 train_2278313361,train_2278313361 train_129225211,"[paper, bag, victoria, secret]",
2,train_2288590299,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,train_2288590299 train_3803689425,train_2288590299 train_129225211,"[maling, tt, canned, pork, luncheon, meat, NUM...",train_2288590299 train_3803689425
3,train_3803689425,Maling Ham Pork Luncheon Meat TTS 397gr,2395904891,train_2288590299 train_3803689425,train_3803689425 train_129225211,"[maling, ham, pork, luncheon, meat, tt, NUM, gr]",
4,train_2406599165,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,train_2406599165 train_3342059966,train_2406599165 train_129225211,"[daster, batik, lengan, pendek, motif, acak, c...",train_2088327894 train_1593362411 train_256062...
...,...,...,...,...,...,...,...
20947,train_1001474240,Sam A20 A30 A30S A50 A50S A51 A71 A70 A70s M10...,3574183172,train_2944131255 train_1001474240,train_1001474240 train_129225211,"[sam, NUM, NUM, NUM, NUM, NUM, NUM, NUM, NUM, ...",
20948,train_2244662893,LAMPU HURUF A-Z DAN ANGKA 0-9 \xe2\x9d\xa4\xef...,763032672,train_2244662893 train_3281898016,train_2244662893 train_129225211,"[lampu, huruf, az, dan, angka, NUM, xe, NUM, x...",train_2244662893 train_3281898016
20949,train_3281898016,LAMPU HURUF A-Z DAN ANGKA 0-9 TINGGI 16 CM,763032672,train_2244662893 train_3281898016,train_3281898016 train_129225211,"[lampu, huruf, az, dan, angka, NUM, tinggi, NU...",
20950,train_4221982820,Sprei Lady Rose 180x200 King terlaris Keroppi,53836859,train_4221982820 train_4063409014,train_4221982820 train_129225211,"[sprei, lady, rose, NUM, x, NUM, king, terlari...",


### 3. TF-IDF

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN

In [20]:
def match_products(data):
    # Извлечение названий товаров из данных
    product_names = [row[2] for row in data.itertuples()]

    # Создание TF-IDF матрицы признаков для названий товаров
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(product_names)
    
    # Вычисление косинусного сходства между парами товаров
    similarity_matrix = cosine_similarity(tfidf_matrix)
    
    # Применение алгоритма кластеризации DBSCAN для объединения одинаковых товаров
    dbscan = DBSCAN(eps=0.5, min_samples=2, metric='precomputed')
    clusters = dbscan.fit_predict(similarity_matrix)
    
    # Сохранение результатов мэтчинга товаров в словаре
    matched_products = {}
    for idx, cluster_id in enumerate(clusters):
        if cluster_id not in matched_products:
            matched_products[cluster_id] = []
        matched_products[cluster_id].append(product_names[idx])
    
    # Оценка модели
    data["prediction_2"] = [cluster_id for cluster_id in clusters]
    return data
#     f1 = f1_score(data['matches'], data["prediction_2"])
#     return f1

In [21]:
f1 = match_products(df)
f1

,posting_id,title,label_group,matches,prediction_0,prediction_2
0,train_129225211,Paper Bag Victoria Secret,249114794,train_129225211 train_2278313361,train_129225211 train_129225211,0
1,train_2278313361,PAPER BAG VICTORIA SECRET,249114794,train_129225211 train_2278313361,train_2278313361 train_129225211,0
2,train_2288590299,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,train_2288590299 train_3803689425,train_2288590299 train_129225211,0
3,train_3803689425,Maling Ham Pork Luncheon Meat TTS 397gr,2395904891,train_2288590299 train_3803689425,train_3803689425 train_129225211,0
4,train_2406599165,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,train_2406599165 train_3342059966,train_2406599165 train_129225211,0
...,...,...,...,...,...,...
20947,train_1001474240,Sam A20 A30 A30S A50 A50S A51 A71 A70 A70s M10...,3574183172,train_2944131255 train_1001474240,train_1001474240 train_129225211,0
20948,train_2244662893,LAMPU HURUF A-Z DAN ANGKA 0-9 \xe2\x9d\xa4\xef...,763032672,train_2244662893 train_3281898016,train_2244662893 train_129225211,0
20949,train_3281898016,LAMPU HURUF A-Z DAN ANGKA 0-9 TINGGI 16 CM,763032672,train_2244662893 train_3281898016,train_3281898016 train_129225211,0
20950,train_4221982820,Sprei Lady Rose 180x200 King terlaris Keroppi,53836859,train_4221982820 train_4063409014,train_4221982820 train_129225211,0
